In [95]:
%pip install langchain-text-splitters transformers pandas numpy tqdm

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [96]:
import numpy as np
import pandas as pd

In [97]:
from transformers import AutoTokenizer
from langchain_text_splitters import CharacterTextSplitter

# Load a tokenizer for a BERT-like model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
def count_tokens(text):
    return len(tokenizer.encode(text))

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=400,
    chunk_overlap=50,
    # length_function=len,
    length_function=count_tokens,
    # is_separator_regex=False,
)

In [98]:
def chunk(text):
    texts = text_splitter.create_documents([text])
    chunks = []
    for i,t in enumerate(texts):
        # replace newlines with spaces this can help keep word boundires
        chunks.append(t.page_content.replace("\n", " "))
    return chunks

In [99]:
df_p = pd.read_csv('./datasets/positive_w_text.csv', delimiter='|')
df_n = pd.read_csv('./datasets/negitive_w_text.csv', delimiter='|')

print(f'Positive Count: {len(df_p)}')
print(f'Negitive Count: {len(df_n)}')

df_p = df_p.drop(columns=["SUBJECT_ID"])
df_n = df_n.drop(columns=["SUBJECT_ID"])

df = pd.concat([df_p, df_n], ignore_index=True)
# df.reset_index(inplace=True)

# This shuffles the dataframe, but in a non-reproducable way
# df = df.sample(frac=1).reset_index(drop=True)
# Instead shuffle dataframe, but in a reproducable way
SEED=42
np.random.seed(SEED)
idx = df.index.to_list()

np.random.shuffle(idx)
df = df.loc[idx].reset_index(drop=True)

# df = df.sample(frac=1).reset_index(drop=True)
display(df)

Positive Count: 500
Negitive Count: 500


,class_number,chart_text
0,0.0,NaN
1,0.0,NaN
2,0.0,NaN
3,0.0,NaN
4,10.0,Bradycardia\n Assessment:\n HR variable th...
...,...,...
995,10.0,Altered mental status (not Delirium)\n Asses...
996,10.0,Hypoglycemia\n Assessment:\n Action:\n R...
997,0.0,"Trauma, s/p ? witnessed fall down grated stair..."
998,10.0,69 y.o. woman adm to ew [**12-13**] p/w resp. ...


In [100]:
from tables import classes, schema_dicts
print(classes)
print(schema_dicts)

['Intellectual Developmental Disorders', 'Communication Disorders', 'Autism Spectrum Disorder', 'Attention-Deficit/Hyperactivity Disorder', 'Specific Learning Disorder', 'Motor Disorders', 'Other Neurodevelo mental Disorders p', 'Schizophrenia Spectrum and Other Psychotic Disorders', 'Catatonia', 'Bipolar and Related Disorders', 'Depressive Disorders', 'Anxiety Disorders', 'Obsessive-Compulsive and Related Disorders', 'Trauma- and Stressor-Related Disorders', 'Dissociative Disorders', 'Somatic Symptom and Related Disorders', 'Feeding and Eating Disorders', 'Elimination Disorders', 'Sleep-Wake Disorders', 'Breathing-Related Sleep Disorders', 'Parasomnias', 'Sexual Dysfunctions', 'Gender Dysphoria', 'Disruptive, Impulse-Control, and Conduct Disorders', 'Neurocognitive Disorders', 'Personality Disorders', 'Cluster A Personality Disorders', 'Cluster B Personality Disorders', 'Cluster C Personalit Disorders y', 'Other Personality Disorders', 'Paraphilic Disorders', 'Other Mental Disorders a

**Schema Index**
| ID | Model | Port |
|----|-------|------|
| 0 | Distilbert | 9090 |
| 1 | Roberta | 9091 |
| 2 | Biobert | 9092 |
| 3 | Clinicalbert | 9093 |

In [ ]:
schema_index = 3
max_avg_distance = 20
test_range = 200

In [102]:
import weaviate
import weaviate.classes as wvc

def query_top_x(text, limit=1, max_distance=20):
    client = weaviate.connect_to_local()
    res = None
    try:
        collection = client.collections.use(schema_dicts[schema_index]["schema"])

        results = collection.query.near_text(
            # Because near_text is a purely vector search, we get a
            # distance and no score
            return_metadata=wvc.query.MetadataQuery(
                score=True, explain_score=True, distance=True, certainty=True
            ),
            query=text,
            limit=limit
        )
        for r in results.objects:
            if r.metadata.distance < max_distance:
                res = [classes.index(r.properties['title']), r.metadata.distance]

        return res
    finally:
        client.close()

In [103]:
def score_document(s, max_dist=10):
    cat = {}
    top_results = []

    for i, _ in enumerate(classes):
        cat[i] = {"count":0, "distance":0}

    for _, d in enumerate(s):
        cat[d[0]]["count"] += 1
        cat[d[0]]["distance"] += d[1]
        cat[d[0]]["avg"] = cat[d[0]]["distance"] / cat[d[0]]["count"]

    for i, _ in enumerate(classes):
        if cat[i]["count"] > 0 and cat[i]["avg"] < max_dist:
            top_results.append([cat[i]["avg"], i])

    return top_results

# top = np.array(score_document(scores, max_avg_distance))
# print(np.sort(top, axis=0))
# print('--------------')
# print(f'{df.iloc[item]["class_number"]:02} {classes[df.iloc[item]["class_number"]]}')

In [104]:
from tqdm import tqdm
results = []
for i in tqdm(range(test_range)):
    text = df.iloc[i]["chart_text"]

    if type(text) == str:
        chart_chunks = chunk(text[:500000])
        # print(f"Chart: {len(text)} \t Chunks: {len(chart_chunks)}")
        scores = []
        for c in chart_chunks:
            d = query_top_x(c, 5)
            if d is not None:
                scores.append(d)

        top = np.array(score_document(scores, max_avg_distance))
        top = np.sort(top, axis=0)

        results.append([i, df.iloc[i]["class_number"], top])

100%|██████████| 200/200 [2:16:06<00:00, 40.83s/it]   


## Scoring

Correct answer in the top 5

In [105]:
df_results = pd.DataFrame(results, columns=["record", "original", "predict"])

def top5(x):
    return np.array([y[1] for i,y in enumerate(x) if i < 5])

def in_top_5(orig, top5):
    return 1 if orig in top5 else 0

def correct_predict(orig, top5):
    if orig == 0 and 11 not in top5 and 10 not in top5:
        return 1
    return 1 if orig in top5 else 0

df_results['top5'] = df_results['predict'].map(top5)
df_results['inTop5'] = df_results.apply(lambda x: in_top_5(orig=x['original'], top5=x['top5']), axis=1)
df_results['correct'] = df_results.apply(lambda x: correct_predict(orig=x['original'], top5=x['top5']), axis=1)

print(f"Correct top 5    {df_results['inTop5'].sum() / test_range}")
print(f"Correct category {df_results['correct'].sum() / test_range}")
display(df_results)

# Sample size 30
# i     %   MMR
# 0   .70  .20
# 1   .60  .16
# 2   .53  .14
# 3   .66  .18


Correct top 5    0.185
Correct category 0.22


,record,original,predict,top5,inTop5,correct
0,4,10.0,"[[0.028194189071655273, 3.0], [0.0287821292877...","[3.0, 5.0, 7.0, 8.0, 9.0]",0,0
1,8,0.0,"[[0.027145830790201823, 3.0], [0.0285262465476...","[3.0, 5.0, 7.0, 8.0, 9.0]",0,1
2,9,10.0,"[[0.029250681400299072, 3.0], [0.0318406422932...","[3.0, 5.0, 7.0, 9.0, 10.0]",1,1
3,11,10.0,"[[0.0328449010848999, 5.0], [0.033925533294677...","[5.0, 18.0, 20.0, 21.0, 23.0]",0,0
4,16,10.0,"[[0.027641206979751587, 3.0], [0.0279233455657...","[3.0, 5.0, 7.0, 10.0, 13.0]",1,1
...,...,...,...,...,...,...
123,194,10.0,"[[0.08009728789329529, 11.0], [0.0977746546268...","[11.0, 23.0, 24.0, 34.0]",0,0
124,195,10.0,"[[0.02814328670501709, 3.0], [0.02993613481521...","[3.0, 9.0, 10.0, 13.0, 19.0]",1,1
125,196,10.0,"[[0.03011232614517212, 5.0], [0.03093928098678...","[5.0, 15.0, 16.0, 17.0, 19.0]",0,0
126,197,10.0,"[[0.028116345405578613, 5.0], [0.0295114219188...","[5.0, 9.0, 13.0, 18.0, 19.0]",0,0


### Mean Reciprocal Rank (MRR)

The position of the first correct result is important

For each query, find the rank of the first correct result. The Reciprocal Rank (RR) is $1 / rank$. MRR is the average RR across all queries.

Example: Suppose you have 3 queries:
- Query 1: Correct result is ranked 1st → RR = 1/1 = 1.0
- Query 2: Correct result is ranked 3rd → RR = 1/3 ≈ 0.33
- Query 3: Correct result is ranked 2nd → RR = 1/2 = 0.5

$$
MRR = (1.0 + 0.33 + 0.5) / 3 ≈ 0.61
$$

$$
MRR = \frac{1}{∣Q∣} \sum_{i=1}^{∣Q∣}\frac{​1}{rank_i}
$$

A good Mean Reciprocal Rank (MRR) value typically ranges from 0.6 to 1.0

In [106]:
def mmr(orig, top5):
    idx = np.where(top5 == orig)[0]
    return (1/(idx+1) if idx.size != 0 else [0])[0]

df_results['mmr'] = df_results.apply(lambda x: mmr(orig=x['original'], top5=x['top5']), axis=1)

print(f"MMR: {df_results['mmr'].sum() / len(df)}")
display(df_results)

MMR: 0.010566666666666667


,record,original,predict,top5,inTop5,correct,mmr
0,4,10.0,"[[0.028194189071655273, 3.0], [0.0287821292877...","[3.0, 5.0, 7.0, 8.0, 9.0]",0,0,0.000000
1,8,0.0,"[[0.027145830790201823, 3.0], [0.0285262465476...","[3.0, 5.0, 7.0, 8.0, 9.0]",0,1,0.000000
2,9,10.0,"[[0.029250681400299072, 3.0], [0.0318406422932...","[3.0, 5.0, 7.0, 9.0, 10.0]",1,1,0.200000
3,11,10.0,"[[0.0328449010848999, 5.0], [0.033925533294677...","[5.0, 18.0, 20.0, 21.0, 23.0]",0,0,0.000000
4,16,10.0,"[[0.027641206979751587, 3.0], [0.0279233455657...","[3.0, 5.0, 7.0, 10.0, 13.0]",1,1,0.250000
...,...,...,...,...,...,...,...
123,194,10.0,"[[0.08009728789329529, 11.0], [0.0977746546268...","[11.0, 23.0, 24.0, 34.0]",0,0,0.000000
124,195,10.0,"[[0.02814328670501709, 3.0], [0.02993613481521...","[3.0, 9.0, 10.0, 13.0, 19.0]",1,1,0.333333
125,196,10.0,"[[0.03011232614517212, 5.0], [0.03093928098678...","[5.0, 15.0, 16.0, 17.0, 19.0]",0,0,0.000000
126,197,10.0,"[[0.028116345405578613, 5.0], [0.0295114219188...","[5.0, 9.0, 13.0, 18.0, 19.0]",0,0,0.000000


In [107]:
df_results.to_csv(f"datasets/{schema_index}_{max_avg_distance}_{test_range}.csv")